# Tutorial IV.V - Modelling the Transportation Problem with JuMP

Applied Optimization with Julia

# Introduction

Welcome to this tutorial on the transportation problem using JuMP! As
always, don’t worry if you’re new to optimization - we’ll walk through
everything step by step using a real-world example.

Imagine you’re running a solar panel distribution company. You have
several warehouses (suppliers) and need to ship solar panels to various
solar farms (customers). Your goal is to minimize the total cost of
transportation while meeting all customer demands.

By the end of this tutorial, you’ll be able to:

1.  Understand what a transportation problem is
2.  Set up a transportation problem using JuMP
3.  Solve the problem and interpret the results

Let’s start by loading the necessary packages:

In [1]:
using JuMP, HiGHS
using DataFrames, CSV

------------------------------------------------------------------------

# Section 1 - Understanding the Transportation Problem

The transportation problem involves:

-   Suppliers (our warehouses)
-   Customers (solar farms)
-   Transportation costs between each supplier and customer
-   Supply available at each warehouse
-   Demand required by each solar farm

Our goal is to decide how many solar panels to ship from each warehouse
to each solar farm to minimize total cost.

Let’s set up our problem:

-   The revenue from each truckload of solar panels is 11000
-   The variable costs from each truckload of solar panels is 6300
-   The available panels at the supplier are given in the file
    `available-panels.csv`
-   The requested panels at the customer are given in the file
    `panel-demand.csv`
-   The transportation costs between suppliers and customers are given
    in the file `cost.csv`

> **Note**
>
> We can use the `CSV.read` function to load the data from a CSV file
> into a DataFrame. If we want to access the directory of the current
> file, we can again use the convinient `@__DIR__` macro.

> **Tip**
>
> Ensure you download the datasets from the website (located below the
> notebook) and save them in a folder named `data` within the same
> directory as your current script. To download the data, simply
> right-click the CSV icon for the desired dataset and choose
> `Download linked file`. No preprocessing is required as this tutorial
> will focus solely on the modeling process.

In [2]:
# Fixed parameters
revenue = 11000  # Revenue per truckload of solar panels
varCosts = 6300  # Variable costs per truckload

# Load data from CSV files
available = CSV.read("$(@__DIR__)/data/available-panels.csv", DataFrame)
requested = CSV.read("$(@__DIR__)/data/panel-demand.csv", DataFrame)
travelCosts = CSV.read("$(@__DIR__)/data/cost.csv", DataFrame)

println("Data loaded successfully!")

Data loaded successfully!


Now, we can check out the data by printing the first few rows of each
DataFrame. We can use the `first` function to get the first few rows of
a DataFrame.

In [3]:
println("Available panels:")
first(available,5)

Available panels:


Row,supplier,truckloads_available
,String7,Int64
1,a_1,478
2,a_2,371
3,a_3,361
4,a_4,415
5,a_5,354


In [4]:
println("Requested panels:")
first(requested,5)

Requested panels:


Row,solar_farm,truckload_demand
,String7,Int64
1,b_1,9
2,b_2,31
3,b_3,47
4,b_4,46
5,b_5,12


In [5]:
println("Travel costs:")
first(travelCosts,5)

Travel costs:


Row,supplier,solar_farm,costs
,String7,String7,Int64
1,a_1,b_1,8052
2,a_2,b_1,11845
3,a_3,b_1,6103
4,a_4,b_1,11099
5,a_5,b_1,5625


## Exercise 1.1 - Understand the Data

Take a moment to look at the data. Can you answer these questions?

1.  How many warehouses do we have? Save the number in a variable called
    `num_warehouses`.
2.  How many solar farms are we supplying? Save the number in a variable
    called `num_solar_farms`.

In [6]:
# YOUR ANSWERS BELOW
# Hint: Use the `nrow()` function to count rows
num_warehouses = nrow(available)
num_solar_farms = nrow(requested)

1000

In [7]:
# Test your understanding
@assert num_warehouses == nrow(available)
@assert num_solar_farms == nrow(requested)

println("Great job! Here are the answers:")
println("Number of warehouses: ", num_warehouses)
println("Number of solar farms: ", num_solar_farms)

Great job! Here are the answers:
Number of warehouses: 100
Number of solar farms: 1000


------------------------------------------------------------------------

# Section 2 - Using dictionaries to store the data

Now, DataFrames are not a very convenient format for our purposes. We
have several options now on how to deal with these data sets. As our
suppliers and customers are given names, it might be useful to convert
the data into dictionaries. Dictionaries are a great data structure that
allow us to store key-value pairs, where the keys are unique identifiers
and the values are the data associated with those keys. By using
dictionaries, we can easily access and modify the data associated with a
specific key.

In [8]:
available_dict = Dict(
    available.supplier .=> available.truckloads_available
)
requested_dict = Dict(
    requested.solar_farm .=> requested.truckload_demand
)
travelCosts_dict = Dict(
    (row.supplier,row.solar_farm) => row.costs
    for row in eachrow(travelCosts)
)

Dict{Tuple{String7, String7}, Int64} with 100000 entries:
  ("a_33", "b_450") => 1903
  ("a_99", "b_340") => 1749
  ("a_74", "b_249") => 7016
  ("a_11", "b_278") => 5788
  ("a_40", "b_35")  => 11369
  ("a_48", "b_457") => 6005
  ("a_69", "b_440") => 9214
  ("a_59", "b_946") => 1103
  ("a_35", "b_880") => 2565
  ("a_96", "b_708") => 4601
  ("a_1", "b_388")  => 8823
  ("a_63", "b_843") => 9774
  ("a_67", "b_576") => 10079
  ("a_30", "b_687") => 1058
  ("a_94", "b_205") => 8087
  ("a_21", "b_445") => 9613
  ("a_43", "b_77")  => 3084
  ("a_36", "b_23")  => 10357
  ("a_26", "b_711") => 1217
  ("a_57", "b_90")  => 3604
  ("a_51", "b_808") => 7695
  ("a_84", "b_960") => 4317
  ("a_32", "b_65")  => 8642
  ("a_6", "b_624")  => 4972
  ("a_62", "b_58")  => 5338
  ⋮                 => ⋮

> **Tip**
>
> You can use the `Dict` function to create a dictionary from two arrays
> or DataFrames. For example: `Dict(keys .=> values)` will create a
> dictionary where the keys are the elements of the `keys` array and the
> values are the elements of the `values` array.

Now, let us check out the dictionaries. We can use the `first` function
to get the first few key-value pairs of a dictionary.

In [9]:
println("Available panels:")
first(available_dict,5)

Available panels:


5-element Vector{Pair{String7, Int64}}:
 "a_13" => 145
 "a_17" => 181
 "a_26" => 405
 "a_90" => 479
 "a_67" => 430

In [12]:
println("Travel costs:")
first(travelCosts_dict,5)

Travel costs:

5-element Vector{Pair{Tuple{String7, String7}, Int64}}:
 ("a_33", "b_450") => 1903
 ("a_99", "b_340") => 1749
 ("a_74", "b_249") => 7016
 ("a_11", "b_278") => 5788
  ("a_40", "b_35") => 11369

Remember, we can also access the value associated with a specific key in
a dictionary by using the key inside square brackets. For example:
`available_dict["a_1"]` will return the value associated with the key
`"a_1"`.

In [10]:
print("Value associated with supplier 'a_1': ")
available_dict["a_1"]

Value associated with supplier 'a_1': 

478

Our travel costs dictionary is a bit more complex, as it is dictionary
with tuples as keys. We can access the value associated with a specific
supplier and customer by using two keys inside square brackets. For
example: `travelCosts_dict[("a_1","b_1")]` will return the value
associated with the keys `("a_1","b_1")`.

In [11]:
print("Value associated with supplier 'a_1' and customer 'b_1': ")
travelCosts_dict[("a_1","b_1")]

Value associated with supplier 'a_1' and customer 'b_1': 

8052

We can also extract the keys and values of a dictionary using the `keys`
and `values` functions, as shown in the previous tutorial.

In [12]:
println("Keys of the travel costs dictionary:")
first(keys(travelCosts_dict),5)

Keys of the travel costs dictionary:


5-element Vector{Tuple{String7, String7}}:
 ("a_33", "b_450")
 ("a_99", "b_340")
 ("a_74", "b_249")
 ("a_11", "b_278")
 ("a_40", "b_35")

In [13]:
println("Values of the travel costs dictionary:")
first(values(travelCosts_dict),5)

Values of the travel costs dictionary:


5-element Vector{Int64}:
  1903
  1749
  7016
  5788
 11369

Dictionaries make it a lot easier to access the data later on, as we can
use the keys to directly access the desired value in our model. This
will be useful when we want to define the constraints later on.

------------------------------------------------------------------------

# Section 3 - The model instance

After the preprocessing and data loading, we now can create the model
instance with the HiGHS optimizer.

## Exercise 3.1 - Creating the model instance

From the last tutorial, you should know how to do this. Create a model
instance called `transport_model` and set the optimizer to HiGHS.

In [14]:
# YOUR CODE BELOW
transport_model = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [15]:
# Test your answer
@assert typeof(transport_model) == JuMP.Model
println("Model instance created successfully!")

Model instance created successfully!


------------------------------------------------------------------------

# Section 4 - Defining the model

## Define the variables

We can now define the variables of our model. We need to define a
variable for each supplier and customer pair. As before, we can use the
`@variable` macro to define the variables. The syntax is
`@variable(model, varname[index1,index2] >= 0)`, where `model` is the
model instance, `varname` is the name of the variable, and `index1` and
`index2` are the indices of the variable. We can use vectors as input
for the indices, but we could also use the keys of the dictionaries. In
the following code block we mixed both options, to show you that it is
possible.

In [16]:
# Define variable
@variable(
    transport_model,
    X[available.supplier,keys(requested_dict)] >= 0
)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, String7["a_1", "a_2", "a_3", "a_4", "a_5", "a_6", "a_7", "a_8", "a_9", "a_10"  …  "a_91", "a_92", "a_93", "a_94", "a_95", "a_96", "a_97", "a_98", "a_99", "a_100"]
    Dimension 2, String7["b_435", "b_727", "b_634", "b_542", "b_434", "b_658", "b_241", "b_230", "b_336", "b_549"  …  "b_123", "b_245", "b_345", "b_553", "b_379", "b_322", "b_143", "b_803", "b_742", "b_724"]
And data, a 100×1000 Matrix{VariableRef}:
 X[a_1,b_435]    X[a_1,b_727]    …  X[a_1,b_742]    X[a_1,b_724]
 X[a_2,b_435]    X[a_2,b_727]       X[a_2,b_742]    X[a_2,b_724]
 X[a_3,b_435]    X[a_3,b_727]       X[a_3,b_742]    X[a_3,b_724]
 X[a_4,b_435]    X[a_4,b_727]       X[a_4,b_742]    X[a_4,b_724]
 X[a_5,b_435]    X[a_5,b_727]       X[a_5,b_742]    X[a_5,b_724]
 X[a_6,b_435]    X[a_6,b_727]    …  X[a_6,b_742]    X[a_6,b_724]
 X[a_7,b_435]    X[a_7,b_727]       X[a_7,b_742]    X[a_7,b_724]
 X[a_8,b_435]    X[a_8,b_727]       X[a_8,b_742]  

## Define the objective

Next, we can define the objective of our model. We want to maximize the
profit, which is the revenue minus the variable costs and the
transportation costs. As before, we can use the `@objective` macro to
define the objective. The syntax is
`@objective(model, Max, expression)`, where `model` is the model
instance, `Max` indicates that we want to maximize the expression, and
`expression` is the expression we want to maximize.

In [17]:
@objective(transport_model, Max,
    sum((revenue-varCosts-travelCosts_dict[(i,j)]) * X[i,j]
    for i in keys(available_dict), j in keys(requested_dict))
)

-2984 X[a_13,b_435] - 4847 X[a_13,b_727] + 1075 X[a_13,b_634] + 884 X[a_13,b_542] + 2825 X[a_13,b_434] - 4052 X[a_13,b_658] + 1214 X[a_13,b_241] - 6772 X[a_13,b_230] + 805 X[a_13,b_336] - 1902 X[a_13,b_549] - 2571 X[a_13,b_811] - 6087 X[a_13,b_529] + 2498 X[a_13,b_360] - 3760 X[a_13,b_467] + 1427 X[a_13,b_418] + 3395 X[a_13,b_276] - 2410 X[a_13,b_561] - 5276 X[a_13,b_980] - 5178 X[a_13,b_447] - 4192 X[a_13,b_599] - 1309 X[a_13,b_6] - 4910 X[a_13,b_442] - 6752 X[a_13,b_619] + 2774 X[a_13,b_924] - 1717 X[a_13,b_606] + 2487 X[a_13,b_841] - 5252 X[a_13,b_674] + 1529 X[a_13,b_55] - 6350 X[a_13,b_505] + 2399 X[a_13,b_648] - [[...99922 terms omitted...]] - 4103 X[a_54,b_369] - 813 X[a_54,b_128] - 2717 X[a_54,b_358] - 5080 X[a_54,b_824] + 818 X[a_54,b_178] - 6773 X[a_54,b_38] - 4408 X[a_54,b_124] - 3999 X[a_54,b_151] - 5287 X[a_54,b_684] + 777 X[a_54,b_929] - 3464 X[a_54,b_215] - 76 X[a_54,b_661] - 4039 X[a_54,b_352] - 2244 X[a_54,b_357] - 6341 X[a_54,b_892] - 2404 X[a_54,b_909] - 3526 X[a_54,

## Define the constraints

We can now define the constraints of our model. We need to ensure that
the supply from each supplier is enough to cover the demand of each
customer. We can use the `@constraint` macro to define the constraints.
The syntax is `@constraint(model, expression)`, where `model` is the
model instance and `expression` is the expression we want to constrain.

To illustrate the use of dictionaries, we will again use the keys of the
dictionaries to define the constraints in the following code block.

In [18]:
@constraint(transport_model,
    restrictAvailable[i in keys(available_dict)],
    sum(X[i,j] for j in keys(requested_dict)) <= available_dict[i]
)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, String7["a_13", "a_17", "a_26", "a_90", "a_67", "a_64", "a_14", "a_2", "a_70", "a_80"  …  "a_10", "a_57", "a_92", "a_86", "a_46", "a_8", "a_74", "a_22", "a_24", "a_54"]
And data, a 100-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 restrictAvailable[a_13] : X[a_13,b_435] + X[a_13,b_727] + X[a_13,b_634] + X[a_13,b_542] + X[a_13,b_434] + X[a_13,b_658] + X[a_13,b_241] + X[a_13,b_230] + X[a_13,b_336] + X[a_13,b_549] + X[a_13,b_811] + X[a_13,b_529] + X[a_13,b_360] + X[a_13,b_467] + X[a_13,b_418] + X[a_13,b_276] + X[a_13,b_561] + X[a_13,b_980] + X[a_13,b_447] + X[a_13,b_599] + X[a_13,b_6] + X[a_13,b_442] + X[a_13,b_619] + X[a_13,b_924] + X[a_13,b_606] +

Naturally, we could also use the vectors with the names from the
DataFrames to define the constraints or we could also just work with
ranges from the beginning, e.g. `1:length(available.supplier)` and
`1:length(requested.solar_farm)`. Working with names is often more
convenient, though.

In [19]:
@constraint(transport_model,
    restrictDemand[j in requested.solar_farm],
    sum(X[i,j] for i in available.supplier) <= requested_dict[j]
)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, String7["b_1", "b_2", "b_3", "b_4", "b_5", "b_6", "b_7", "b_8", "b_9", "b_10"  …  "b_991", "b_992", "b_993", "b_994", "b_995", "b_996", "b_997", "b_998", "b_999", "b_1000"]
And data, a 1000-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 restrictDemand[b_1] : X[a_1,b_1] + X[a_2,b_1] + X[a_3,b_1] + X[a_4,b_1] + X[a_5,b_1] + X[a_6,b_1] + X[a_7,b_1] + X[a_8,b_1] + X[a_9,b_1] + X[a_10,b_1] + X[a_11,b_1] + X[a_12,b_1] + X[a_13,b_1] + X[a_14,b_1] + X[a_15,b_1] + X[a_16,b_1] + X[a_17,b_1] + X[a_18,b_1] + X[a_19,b_1] + X[a_20,b_1] + X[a_21,b_1] + X[a_22,b_1] + X[a_23,b_1] + X[a_24,b_1] + X[a_25,b_1] + X[a_26,b_1] + X[a_27,b_1] + X[a_28,b_1] + X[a_29,b_1] +

And that’s it! We have now defined the model and can start optimizing.

------------------------------------------------------------------------

# Section 5 - Solving the model

## Exercise 5.1 - Start optimization

Start the optimization as usual by calling the `optimize!` function on
the model instance.

In [20]:
# YOUR CODE BELOW
optimize!(transport_model)

Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 1100 rows; 100000 cols; 200000 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [1e+00, 7e+03]
  Bound   [0e+00, 0e+00]
  RHS     [6e+00, 5e+02]
Presolving model
1100 rows, 33689 cols, 67378 nonzeros  0s
1100 rows, 33689 cols, 67378 nonzeros  0s
Presolve reductions: rows 1100(-0); columns 33689(-66311); nonzeros 67378(-132622) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.1896193553e+07 Ph1: 1100(67378); Du: 33689(6.18962e+07) 0s
       3013    -1.0037340600e+08 Pr: 0(0) 1s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 3013
Objective value     :  1.0037340600e+08
P-D objective error :  0.0000000000e+00
HiGHS run time      :          1.13


In [21]:
# Test your answer
@assert termination_status(transport_model) == MOI.OPTIMAL
println("Model optimized successfully!")

Model optimized successfully!


Now, we can access the values of the variables at the optimal solution.
But remember, we defined the variables with the keys of the
dictionaries, so we need to convert the result back to a DataFrame.
Calling the variable itself will just show the structure of the
variable, not the values.

In [22]:
first(X,5)

5-element Vector{VariableRef}:
 X[a_1,b_435]
 X[a_2,b_435]
 X[a_3,b_435]
 X[a_4,b_435]
 X[a_5,b_435]

Thus, we need to use the `value` function to extract the values from the
variable.

In [23]:
transport_values = value.(X)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, String7["a_1", "a_2", "a_3", "a_4", "a_5", "a_6", "a_7", "a_8", "a_9", "a_10"  …  "a_91", "a_92", "a_93", "a_94", "a_95", "a_96", "a_97", "a_98", "a_99", "a_100"]
    Dimension 2, String7["b_435", "b_727", "b_634", "b_542", "b_434", "b_658", "b_241", "b_230", "b_336", "b_549"  …  "b_123", "b_245", "b_345", "b_553", "b_379", "b_322", "b_143", "b_803", "b_742", "b_724"]
And data, a 100×1000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0   0.0  0.0  …  0.0  0.0  0.0  43.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0   0.0  0.0  …  0.0  0.0  0.0   0.0  0.0  7.0  0.0
 0.0  0.0  0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0  0.

The result is a `DenseAxisArray{Float64,2,...}` with index sets. To
convert it to a DataFrame, we just need to iterate over the keys
dictionaries and store the values in a new DataFrame. As we are not
interested in values which are zero, we can skip those.

First, we need to initialize an empty DataFrame with the correct column
names.

In [24]:
transport_df = DataFrame(
    supplier = [],
    solar_farm = [],
    truckloads = []
)

Row,supplier,solar_farm,truckloads
,Any,Any,Any


Then, we can iterate over the keys of the dictionaries and store the
values in the DataFrame if they are greater than zero.

In [25]:
for i in keys(available_dict)
    for j in keys(requested_dict)
        if transport_values[i,j] > 0
            push!(transport_df, (
                supplier = i,
                solar_farm = j,
                truckloads = transport_values[i,j]
                )
            )
        end
    end
end

Finally, we can print the first few rows of the transportation plan to
check if it looks correct.

In [26]:
println("Begining of the transportation plan:")
first(transport_df,5)

Begining of the transportation plan:


Row,supplier,solar_farm,truckloads
,Any,Any,Any
1,a_13,b_493,38.0
2,a_13,b_407,7.0
3,a_13,b_898,32.0
4,a_13,b_988,15.0
5,a_13,b_959,7.0


> **Note**
>
> Although the above code looks rather complicated, it is essentially
> just iterating over the keys of the dictionaries and storing the
> values in a new DataFrame. This is a common pattern in optimization,
> as we often want to convert the result of an optimization problem into
> a more convenient format for reporting or further processing.

------------------------------------------------------------------------

# Conclusion

In this tutorial, we have learned how to model and solve the
transportation problem using JuMP. We have also learned how to use
dictionaries to store and access the data, which will be useful for more
complex models. If you have any questions, feel free to ask me via
email!

# Solutions

You will likely find solutions to most exercises online. However, I
strongly encourage you to work on these exercises independently without
searching explicitly for the exact answers to the exercises.
Understanding someone else’s solution is very different from developing
your own. Use the lecture notes and try to solve the exercises on your
own. This approach will significantly enhance your learning and
problem-solving skills.

Remember, the goal is not just to complete the exercises, but to
understand the concepts and improve your programming abilities. If you
encounter difficulties, review the lecture materials, experiment with
different approaches, and don’t hesitate to ask for clarification during
class discussions.